# Convert SQLite outputs to parquet files with cytotable

## Import libraries

In [1]:
import pathlib
import pandas as pd

# cytotable will merge objects from SQLite file into single cells and save as parquet file
from cytotable import convert, presets

import logging

# Set the logging level to a higher level to avoid outputting unnecessary errors from config file in convert function
logging.getLogger().setLevel(logging.ERROR)

## Set paths and variables

In [2]:
# preset configurations based on typical CellProfiler outputs
preset = "cellprofiler_sqlite_pycytominer"

# update preset to include site metadata and cell counts
joins = presets.config["cellprofiler_sqlite_pycytominer"]["CONFIG_JOINS"].replace(
    "Image_Metadata_Well,",
    "Image_Metadata_Well, Image_Metadata_Site, Image_Count_Cells,",
)

# type of file output from cytotable (currently only parquet)
dest_datatype = "parquet"

# set path to directory with SQLite files
sqlite_dir = pathlib.Path("../2.cellprofiler_processing/cp_output")

# directory for processed data
output_dir = pathlib.Path("data")
output_dir.mkdir(parents=True, exist_ok=True)

plate_names = []

for file_path in sqlite_dir.iterdir():
    plate_names.append(file_path.stem)

# print the plate names and how many plates there are (confirmation)
print(f"There are {len(plate_names)} plates in this dataset. Below are the names:")
for name in plate_names:
    print(name)

There are 4 plates in this dataset. Below are the names:
localhost230405150001
localhost220512140003_KK22-05-198
localhost231120090001
localhost220513100001_KK22-05-198_FactinAdjusted


## Convert SQLite to parquet files

In [3]:
for file_path in sqlite_dir.iterdir():
    output_path = pathlib.Path(f"{output_dir}/converted_profiles/{file_path.stem}_converted.parquet")
    print("Starting conversion with cytotable for plate:", file_path.stem)
    # merge single cells and output as parquet file
    convert(
        source_path=str(file_path),
        dest_path=str(output_path),
        dest_datatype=dest_datatype,
        preset=preset,
        joins=joins,
        chunk_size=5000
    )
    
print("All plates have been converted with cytotable!")

Starting conversion with cytotable for plate: localhost231120090001
All plates have been converted with cytotable!


# Load in converted profiles to update

In [4]:
# Directory with converted profiles
converted_dir = pathlib.Path(f"{output_dir}/converted_profiles")

for file_path in converted_dir.iterdir():
    # Load the DataFrame from the Parquet file
    df = pd.read_parquet(file_path)

    # If any, drop rows where "Metadata_ImageNumber" is NaN (artifact of cytotable)
    df = df.dropna(subset=["Metadata_ImageNumber"])

    # Columns to move to the front
    columns_to_move = ['Nuclei_Location_Center_X', 'Nuclei_Location_Center_Y', 'Cells_Location_Center_X', 'Cells_Location_Center_Y', "Image_Count_Cells"]

    # Rearrange columns and add "Metadata" prefix in one line
    df = (df[columns_to_move + [col for col in df.columns if col not in columns_to_move]]
                .rename(columns=lambda col: 'Metadata_' + col if col in columns_to_move else col))

    # Save the processed DataFrame as Parquet in the same path
    df.to_parquet(file_path, index=False)

## Check output to confirm process worked

To confirm the number of single cells is correct, please use any database browser software to see if the number of rows in the "Per_Cells" compartment matches the number of rows in the data frame.

In [5]:
converted_df = pd.read_parquet("./data/converted_profiles/localhost231120090001_converted.parquet")

print(converted_df.shape)
converted_df.head()

(21530, 2016)


,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Image_Metadata_Plate,Image_Metadata_Site,Image_Metadata_Well,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mitochondria_3_00_256,Nuclei_Texture_Variance_Mitochondria_3_01_256,Nuclei_Texture_Variance_Mitochondria_3_02_256,Nuclei_Texture_Variance_Mitochondria_3_03_256,Nuclei_Texture_Variance_PM_3_00_256,Nuclei_Texture_Variance_PM_3_01_256,Nuclei_Texture_Variance_PM_3_02_256,Nuclei_Texture_Variance_PM_3_03_256
0,199.395062,268.792593,193.914743,269.641581,18,1.0,localhost231120090001,f00,B02,4.0,...,2.682473,2.711816,0.417965,0.413393,0.391549,0.385171,1.871388,1.714265,1.808903,1.785609
1,303.540675,350.732143,314.842955,368.939934,18,1.0,localhost231120090001,f00,B02,6.0,...,2.827510,2.799884,0.255611,0.251139,0.280049,0.265307,0.915498,0.914829,0.930071,0.907815
2,700.388506,371.800000,708.112277,380.711178,18,1.0,localhost231120090001,f00,B02,7.0,...,0.867296,0.888812,0.182717,0.181873,0.190170,0.183415,0.789052,0.799555,0.775300,0.771433
3,855.869318,481.075758,877.161009,515.307396,18,1.0,localhost231120090001,f00,B02,8.0,...,2.680820,2.670847,0.379754,0.378857,0.363841,0.348069,2.737748,2.866280,2.822142,2.630111
4,586.624514,509.892023,590.453967,504.113587,18,1.0,localhost231120090001,f00,B02,9.0,...,3.523949,3.555939,0.160312,0.171782,0.161702,0.159216,1.310827,1.325847,1.319233,1.259646
